# Defining cell types by their electrophysiology

### The brain has thousands of different types of cells. How do we even begin to tease them apart?

We can define neurons by their <b>gene expression patterns</b>, <b>electrophysiology features</b>, and <b>structure</b>. Here, we'll use these features to compare and contrast cell types in the brain.

This notebook will help us investigate specific features in the electrophysiology dataset from the Allen Brain Atlas.

### Before diving into this notebook:
- Complete [the web based activity](https://sites.google.com/ucsd.edu/neuroedu/cell-types/web-instructions?authuser=0), which asks you to look at this data on the Allen Institute website.
- Work through [this Introduction to Jupyter Notebooks tutorial](https://github.com/ajuavinett/CellTypesLesson/blob/master/Introduction%20to%20Jupyter%20Notebooks.ipynb) which will introduce you to Python and the Jupyter Notebook environment.

If you'd like to learn a bit more coding, you should check switch to the [`CodingFocused` version of this notebook](https://github.com/ajuavinett/CellTypesLesson/blob/master/CellTypesNotebook-CodingFocused.ipynb).

## Tutorial Steps
1. [Set up our coding environment](#setup)

**Data from a single cell**. *Steps 2-4 are an optional demonstration of the same cell from the [web-based activity](https://sites.google.com/ucsd.edu/neuroedu/cell-types?authuser=0). These steps can be skipped.*

2. [Import data for a single cell](#import)
3. [Plot a raw sweep of data](#plotsweep)
4. [Plot the morphology of the cell](#morphology)

**Data from many cells**. *Steps 5-7 will help you look at pre-computed features for all of the cells in the database.*

5. [Analyze computed features](#metrics)
6. [Compare action potential waveforms](#waveforms)
7. [Compare cell types](#compare)
<hr>

<a id="setup"></a>
## Step 1. Set up our coding environment
Whenever we start an analysis in Python, we need to be sure to import the necessary code packages. If you're running this notebook on Colab or Binder, the cells below will install packages into your coding environment -- we are *not* installing anything on your computer.

### Install the AllenSDK
The Allen Institute has compiled a set of code and tools called a **Software Development Kit** (SDK). These tools will help us import and analyze the cell types data. See [Technical Notes](#technical) at the end of this notebook for more information about working with the AllenSDK.

><b>Task</b>: Run the cell below, which will install the allensdk into your coding environment.

**Technical notes about installing the allensdk**
- If you're running this in Colab, you'll also be prompted to **Restart Runtime** after this is completed. Click the **Restart Runtime** button (nothing will happen), and then you're ready to proceed.
- If you receive an error or are running this notebook on your local computer, there are additional instructions on how to install the SDK locally <a href="http://alleninstitute.github.io/AllenSDK/install.html">here</a>.

In [ ]:
# This will ensure that the AllenSDK is installed.
try:
    import allensdk
    print('allensdk imported')
except ImportError as e:
    !pip install allensdk "numpy == 1.26.4" "pandas == 2.3.0" "matplotlib > 3.8.0" "statsmodels >= 0.14.4"

# you can check the version of the allensdk

allensdk imported


In [ ]:
# IF YOU USE COLAB, run this cell
# it will give access for colab to access your google drive, so you don't have to download all the data multiple times.
# give access only to your google drive files and folders, others are not necessary

# Step 1
# Setup and add files needed to access gdrive
from google.colab import drive                                   # these lines mount your gdrive to access the files we import below
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# this code creates a folder where you can save your data. This should be either on your computer or on your google drive, depending on
# where you run this code.
import os
from pathlib import Path

save_path = '/content/gdrive/MyDrive/Analyzing-Open-Neuroscience-Data-Course-SZTE/'   # set this to wherever you want to save your data

# check if folder already exists; if not, make it
# if it already exists, let us know
if os.path.isdir(save_path):
   print("Given directory exists")
else:
   print("Given directory doesn't exist, making it")
   Path(save_path).mkdir(parents=True,exist_ok=True)# make the folder so it exists

# change into the folder for the session and use it to save our figures
os.chdir(save_path)
print("Changed into directory")

print('data will be saved to {}'.format(save_path))

# check out your google drive!!

Given directory doesn't exist, making it
Changed into directory
data will be saved to /content/gdrive/MyDrive/Analyzing-Open-Neuroscience-Data-Course-SZTE/


### Import common packages
Below, we'll `import` a common selection of packages that will help us analyze and plot our data. We'll also configure the plotting in our notebook.
><b>Task</b>: Just like you did in the [introductory notebook](https://github.com/ajuavinett/CellTypesLesson/blob/master/Introduction%20to%20Jupyter%20Notebooks.ipynb), import the numpy module nicknamed as <code>np</code>. Add a <code>print</code> message at the end that says "Packages imported!" so that you know the code ran.

In [ ]:
# Import our plotting package from matplotlib
import matplotlib.pyplot as plt

# Specify that all plots will happen inline & in high resolution
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Import pandas for working with databases
import pandas as pd

# Import numpy below

# Add your print() statement below


In [ ]:
# you will also need this helper function on colab, please run it

import h5py
def get_sweep(self, sweep_number):
  """ Retrieve the stimulus, response, index_range, and sampling rate
  for a particular sweep.  This method hides the NWB file's distinction
  between a "Sweep" and an "Experiment".  An experiment is a subset of
  of a sweep that excludes the initial test pulse.  It also excludes
  any erroneous response data at the end of the sweep (usually for
  ramp sweeps, where recording was terminated mid-stimulus).

  Some sweeps do not have an experiment, so full data arrays are
  returned.  Sweeps that have an experiment return full data arrays
  (include the test pulse) with any erroneous data trimmed from the
  back of the sweep.

  Parameters
  ----------
  sweep_number: int

  Returns
  -------
  dict
      A dictionary with 'stimulus', 'response', 'index_range', and
      'sampling_rate' elements.  The index range is a 2-tuple where
      the first element indicates the end of the test pulse and the
      second index is the end of valid response data.
  """
  with h5py.File(self.file_name, 'r') as f:

      swp = f['epochs']['Sweep_%d' % sweep_number]

      # fetch data from file and convert to correct SI unit
      # this operation depends on file version. early versions of
      #   the file have incorrect conversion information embedded
      #   in the nwb file and data was stored in the appropriate
      #   SI unit. For those files, return uncorrected data.
      #   For newer files (1.1 and later), apply conversion value.
      major, minor = self.get_pipeline_version()
      if (major == 1 and minor > 0) or major > 1:
          # stimulus
          stimulus_dataset = swp['stimulus']['timeseries']['data']
          conversion = float(stimulus_dataset.attrs["conversion"])
          stimulus = stimulus_dataset[()] * conversion
          # acquisition
          response_dataset = swp['response']['timeseries']['data']
          conversion = float(response_dataset.attrs["conversion"])
          response = response_dataset[()] * conversion
      else:   # old file version
          stimulus_dataset = swp['stimulus']['timeseries']['data']
          stimulus = stimulus_dataset[()]
          response = swp['response']['timeseries']['data'][()]

      if 'unit' in stimulus_dataset.attrs:
          unit = stimulus_dataset.attrs["unit"].decode('UTF-8')

          unit_str = None
          if unit.startswith('A'):
              unit_str = "Amps"
          elif unit.startswith('V'):
              unit_str = "Volts"
          assert unit_str is not None, Exception(
              "Stimulus time series unit not recognized")
      else:
          unit = None
          unit_str = 'Unknown'

      swp_idx_start = swp['stimulus']['idx_start'][()]
      swp_length = swp['stimulus']['count'][()]

      swp_idx_stop = swp_idx_start + swp_length - 1
      sweep_index_range = (swp_idx_start, swp_idx_stop)

      # if the sweep has an experiment, extract the experiment's index
      # range
      try:
          exp = f['epochs']['Experiment_%d' % sweep_number]
          exp_idx_start = exp['stimulus']['idx_start'][()]
          exp_length = exp['stimulus']['count'][()]
          exp_idx_stop = exp_idx_start + exp_length - 1
          experiment_index_range = (exp_idx_start, exp_idx_stop)
      except KeyError:
          # this sweep has no experiment.  return the index range of the
          # entire sweep.
          experiment_index_range = sweep_index_range

      assert sweep_index_range[0] == 0, Exception(
          "index range of the full sweep does not start at 0.")

      return {
          'stimulus': stimulus,
          'response': response,
          'stimulus_unit' : unit_str,
          'index_range': experiment_index_range,
          'sampling_rate': 1.0 * swp['stimulus']['timeseries']['starting_time'].attrs['rate']
      }

### Import the CellTypesModule from the allensdk
With the allensdk installed, we can `import` the **CellTypesCache module**.

The CellTypesCache that we're importing provides tools to allow us to get information from the cell types database. We're giving it a **manifest** filename as well. CellTypesCache will create this manifest file, which contains metadata about the cache. If you want, you can look in the cell_types folder in your code directory and take a look at the file.

><b>Task</b>: Run the cell below. If you'd like a slightly faster way of working with the raw data files required by this notebook, you can also **uncomment** (remove the `#`) from the `!git` and `%cd` lines below to clone the data from Github.

In [ ]:
#Import the "Cell Types Cache" from the AllenSDK core package
from allensdk.core.cell_types_cache import CellTypesCache

#Initialize the cache as 'ctc' (cell types cache)
ctc = CellTypesCache(manifest_file='cell_types/manifest.json') #

print('CellTypesCache imported.')

# check out what happened in your google drive folder

<a id="import"></a>

## Step 2. Import data for a single cell
**Note**:  *Steps 2-4 are an optional demonstration of the same cell from the [web-based activity](https://sites.google.com/ucsd.edu/neuroedu/cell-types?authuser=0). These steps can be skipped if you'd like to dive into analyzing populations of cells below.*

Now that we have the module that we need, let's import a raw sweep of the data. The cell below will grab the data for the same experiment you just looked at on the website. This data is in the form of a [**Neuroscience Without Borders** (NWB)](https://www.nwb.org/) file.

><b>Task</b>: Find the cell specimen ID for the first cell you looked at on the Allen Institute website (hint: it's in the URL), and assign this to <code>cell_id</code> below by replacing the <code>...</code>. Run the cell.

This might take a minute or two. You should wait until the circle in the upper right is <i>not</i> filled (Jupyter Notebook) or you have a green checkmark (Colab) to continue.

<details>
<summary> <font color='blue'>If you didn't do the web-based activity...</font></summary>
    
Use this cell ID: 474626527.
    
</details>

In [ ]:
# Enter your cell_id below
cell_id = ...

# Get the electrophysiology (ephys) data for that cell
data = ctc.get_ephys_data(cell_id)
sweep_data = ctc.get_ephys_sweeps(cell_id)

print('Data retrieved')

# what happened? check your drive!

Thankfully, our NWB file has some built-in **methods** to enable us to pull out a recording sweep. We can access methods of objects like our `data` object by adding a period, and then the method. That's what we would be doing below, with `data.get_sweep()`.

Unfortunately, this method doesn't work as it should on Colab, so this is why we are using the get_sweep() function instead, that we defined above.


><b>Task:</b> Choose your favorite sweep below. (<u>Hint</u>: go back to the website to see what the sweep numbers are.)
    
<details>
<summary> <font color='blue'>If you didn't do the web-based activity...</font></summary>
    
Use sweep ID 28.
    
</details>

In [ ]:
# Assign your favorite sweep number to a variable "sweep_number" below.
import allensdk
sweep_number = ...

sweep_data = get_sweep(data,sweep_number)
print('Sweep obtained')

<a id="plotsweep"></a>
## Step 3. Plot a raw sweep of data
Now that you've pulled down some data, chosen a cell, and chosen a sweep number, let's plot that data.

><b>Task:</b> Run the cell below!

<i>Note</i>: There's a lot of code here, but don't worry about it too much. The principles are the same as in the `Introduction to Jupyter Notebooks`, except now we're plotting "subplots" -- multiple plots within one image -- so the code is a little bit more complicated.

In [ ]:
# Get the stimulus trace (in amps) and convert to pA
stim_current = sweep_data['stimulus'] * 1e12

# Get the voltage trace (in volts) and convert to mV
response_voltage = sweep_data['response'] * 1e3

# Get the sampling rate and can create a time axis for our data
sampling_rate = sweep_data['sampling_rate'] # in Hz
timestamps = (np.arange(0, len(response_voltage)) * (1.0 / sampling_rate))

# Set up our plot
fig, axes = plt.subplots(2, 1, sharex=True,figsize=(8,6))

# axes 0 is our first plot, of the recorded voltage data
axes[0].plot(timestamps, response_voltage, color='black')
axes[0].set_ylabel('mV')
# axes[0].set_xlim(0,3) # determines the scaling of the x-axis
axes[0].set_title('whole-cell patch recording')

# axes 1 is our second plot, of the stimulus trace
axes[1].plot(timestamps, stim_current, color='gray')
axes[1].set_ylabel('pA')
axes[1].set_xlabel('seconds')
axes[1].set_title('stimulus')

plt.show()

This plot probably looks different from the plot on the website, why?

><b>Task:</b> There is a line that has been commented out above.
- Uncomment the line (remove the `#` in front of it) to change the scaling of the x-axis in order to zoom in on the period where the current was applied.
- You can change the values within the parentheses after <code>set_xlim</code> to change the extent of the x axis. There is more documentation about this method <a href="https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.set_xlim.html">here</a>.
- Re-run the cell to re-plot the data.

<a id="morphology"></a>
## Step 4. Plot the morphology of the cell
The Cell Types Database also contains **3D reconstructions** of neuronal morphologies. Here, we'll plot the reconstruction of our cell's morphology.

*Note*: It will take several minutes to run the cell below, possibly longer over a slow internet connection. Running this cell is optional and can be skipped if necessary.

In [ ]:
# Import necessary toolbox
from allensdk.core.swc import Marker

# Download and open morphology and marker files
morphology = ctc.get_reconstruction(cell_id)
markers = ctc.get_reconstruction_markers(cell_id)

# Set up our plot
fig, axes = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(10,10))
axes[0].set_aspect('equal')
axes[1].set_aspect('equal')

# Make a line drawing of x-y and y-z views
for n in morphology.compartment_list:
    for c in morphology.children_of(n):
        axes[0].plot([n['x'], c['x']], [n['y'], c['y']], color='black')
        axes[1].plot([n['z'], c['z']], [n['y'], c['y']], color='black')

# cut dendrite markers
dm = [ m for m in markers if m['name'] == Marker.CUT_DENDRITE ]
axes[0].scatter([m['x'] for m in dm], [m['y'] for m in dm], color='#3333ff')
axes[1].scatter([m['z'] for m in dm], [m['y'] for m in dm], color='#3333ff')

# no reconstruction markers
nm = [ m for m in markers if m['name'] == Marker.NO_RECONSTRUCTION ]
axes[0].scatter([m['x'] for m in nm], [m['y'] for m in nm], color='#333333')
axes[1].scatter([m['z'] for m in nm], [m['y'] for m in nm], color='#333333')
axes[0].set_ylabel('y')
axes[0].set_xlabel('x')
axes[1].set_xlabel('z')
plt.show()

<a id="metrics"></a>
## Step 5. Analyze pre-computed features

The Cell Types Database contains a set of features that have already been computed, which could serve as good starting points for analysis. We can query the database to get these features. Below, we'll use the Pandas package that we imported above to create a **dataframe** for our data.

><b>Task</b>: Run the cell below. It'll take ~10 seconds. It will print a list of all of the feature available, as well as produce a dataframe, which looks something like an excel spreadsheet. You can scroll to the right to see many of the different features available in this dataset.

In [ ]:
# Download all electrophysiology features for all cells
ephys_features = ctc.get_ephys_features()

# what is ephys_features? (type?)
# what is each entry?

#can you convert it to something else so it would be easier to look at?


In [ ]:
ef_df = pd.DataFrame(ephys_features).set_index('specimen_id')

print('Ephys features available for %d cells:' % len(ef_df))
ef_df.head() # Just show the first 5 rows (the head) of our dataframe

As you can see in the dataframe above, there are many **pre-computed features** available in this dataset. [Here's a glossary](https://docs.google.com/document/d/1YGLwkMTebwrXd_1E817LFbztMjSTCWh83Mlp3_3ZMEo/edit?usp=sharing), in case you're curious.

![](https://github.com/ajuavinett/CellTypesLesson/blob/master/docs/ap_features.png?raw=true)
Image from the <a href="http://help.brain-map.org/download/attachments/8323525/CellTypes_Ephys_Overview.pdf">Allen Institute Cell Types Database Technical Whitepaper.</a>
<br><br>
Let's first look at the speed of the trough, and the ratio between the upstroke and downstroke of the action potential.

- <b>Action potential fast trough</b>(<code>fast_trough_v_long_square</code>): Minimum value of the membrane potential in the interval lasting 5 ms after the peak.
- <b>Upstroke/downstroke ratio</b> (<code>upstroke_downstroke_ratio_long_square</code>): The ratio between the absolute values of the action potential peak upstroke and the action potential peak downstroke.

Below, we'll plot the values of the trough vs. the upstroke-downstroke ratio in a scatterplot.

><b>Task</b>: Run the cell below.

In [ ]:
plt.scatter(ef_df['fast_trough_v_long_square'],
            ef_df['upstroke_downstroke_ratio_long_square'])
plt.ylabel("upstroke-downstroke ratio")
plt.xlabel("fast trough depth (mV)")
plt.show()

It looks like there may be roughly two clusters in the data above. Maybe they relate to whether the cells are presumably excitatory (spiny) cells or inhibitory (aspiny) cells. Let's query the API and split up the two sets to see.

><b>Task:</b> The cell below will dig up the dendrite type of these cells and add that to our dataframe. Then, it'll create our same scatterplot, where each dot is colored by dendrite type. All you need to do is run the cell! Right click to save the image as 'scatterplot.png' once it generates.

In [ ]:
# Get information about our cells' dendrites
cells = ctc.get_cells()

# what is the type of the variable cells?
# ANSWER:

#can you make it a pandas dataframe?


In [ ]:
full_dataframe = ef_df.join(pd.DataFrame(cells).set_index('id'))

# Create a dataframe for spiny cells, and a dataframe for aspiny cells
spiny_df = full_dataframe[full_dataframe['dendrite_type'] == 'spiny']
aspiny_df = full_dataframe[full_dataframe['dendrite_type'] == 'aspiny']

# Create our plot! Calling scatter twice like this will draw both of these on the same plot.
plt.scatter(spiny_df['fast_trough_v_long_square'],spiny_df['upstroke_downstroke_ratio_long_square'])
plt.scatter(aspiny_df['fast_trough_v_long_square'],aspiny_df['upstroke_downstroke_ratio_long_square'])

plt.ylabel('upstroke-downstroke ratio')
plt.xlabel('fast trough depth (mV)')
plt.legend(['Spiny','Aspiny'])

plt.show()

Looks like these two clusters do partially relate to the dendritic type. Cells with spiny dendrites (which are typically excitatory cells) have a big ratio of upstroke:downstroke, and a more shallow trough (less negative). Cells with aspiny dendrites (typically inhibitory cells) are a little bit more varied. But </i>only</i> aspiny cells have a low upstroke:downstroke ratio and a deeper trough (more negative).

In [ ]:
#TASK:
# instead of the scatter plot, prepare histograms of each variable and look at the distributions

<a id="waveforms"></a>

## Step 6. Compare waveforms
Let's take a closer look at the action potentials of these cells to see what these metrics actually mean for the action potential waveform by choosing one of the cells with the highest upstroke:downstroke ratio.

><b>Task</b>: Run the cell below. The first line will sort our dataframe so that it's descending (<code>ascending=False</code>) based on upstroke:downstroke ratio. This is an example of a **boolean** in Python. You can change this to say `ascending=True` if you want to sort with *lowest* ratio at the top.

In [ ]:
# Sort the dataframe and reassign
ef_df_upstroke_sorted = ef_df.sort_values('upstroke_downstroke_ratio_long_square',ascending=False)

# Assign one of the top cells in our dataframe (default = 2) and the ratio to different variables
specimen_id = ef_df_upstroke_sorted.index[2]
ratio = ef_df_upstroke_sorted.iloc[2]['upstroke_downstroke_ratio_long_square']

# Print our results so that we can see them
print('Specimen ID: ' + str(specimen_id) + ' with upstroke-downstroke ratio: ' + str(ratio))

Now we can take a closer look at the action potential for that cell by grabbing a raw sweep of recording from it, just like we did above.

><b>Task:</b> Run the cell below. This may take a minute or so.

In [ ]:
# Get the data for our specimen
upstroke_data = ctc.get_ephys_data(specimen_id)

# Get one sweep for our specimen (I've already handselected a gorgeous one for you, 45)
upstroke_sweep = get_sweep(upstroke_data,45)

# Get the current & voltage traces
current = upstroke_sweep['stimulus'] * 1e12 # in A, converted to pA
voltage = upstroke_sweep['response'] * 1e3 # converted to mV!

# Get the time stamps for our voltage trace
timestamps = (np.arange(0, len(voltage)) * (1.0 / upstroke_sweep['sampling_rate']))

print('Sweep obtained')

And now let's plot our sweep. You may need to change some values in the plotting script to actually zoom in on the action potential, or change the sweep number (set to 45 by default) to one where you can see an action potential.

><b>Task</b>: Modify the plotting script below so that you can actually see the shape of the action potential, and then run. <i>Hint</i>: Check out how we did this for the raw traces above.  Right click to save your image as 'high_upstroke.png' when you're content with it.

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(8,6),sharex=True)

# axes 0 is our first plot, of the recorded voltage data
axes[0].plot(timestamps, voltage, color='black')
axes[0].set_ylabel('mV')
axes[0].set_title('whole-cell patch recording')

#axes 1 is our second plot, of the stimulus trace
axes[1].plot(timestamps, current, color='gray')
axes[1].set_ylabel('pA')
axes[1].set_xlabel('seconds')
axes[1].set_title('stimulus')

plt.show()

Let's do the same thing for a cell with a low upstroke ratio and compare.

><b>Task:</b> Edit the line several blocks above that assigns the variable <code>ef_df_upstroke_sorted</code>. Where it says <code>ascending=</code>, set it to <code>True</code>, instead of <code>False</code>. Once you do that, re-run that cell and the cells below to re-plot the data for a new cell with a <i>low</i> upstroke-downstroke ratio. Right click to save your image as 'low_upstrokeratio.png' when you're done.

As you can see, even that one metric, upstroke:downstroke ratio, means the shape of the action potential is dramatically different. The other metric above, size of the trough, is highly correlated with upstroke:downstroke. You can see that by comparing the two cells here. Cells with high upstroke:downstroke tend to have less negative troughs (undershoots) after the action potential.

<a id="compare"></a>

## Step 7. Compare cell types
Let's get out of the action potential weeds a bit. What if we want to know a big picture thing, such as are *human cells different than mouse cells?* Or *how are excitatory cells different from inhibitory cells?*  To ask these questions, we can pull out the data for two different cell types, defined by their species, dendrite type, or transgenic line.

**About Transgenic Cre Lines.** The Allen Institute for Brain Science uses transgenic mouse lines that have Cre-expressing cells to mark specific types of cells in the brain. This technology is called the **Cre-Lox system**, and is a common way in neuroscience (and some other fields) to target cells based on their expression of specific genetic promotors. For more information about Cre/Lox technology, see [this website](https://old.abmgood.com/marketing/knowledge_base/Cre-Lox_Recombination.php). Information about the different Cre lines that are available can be found in [this glossary](https://docs.google.com/document/d/1ZMMZgc7cS5BHhoWNqzjw95BdxOuj5wrYl9I7PV2HeUI/edit?usp=sharing) or on the [Allen Institute's website](http://connectivity.brain-map.org/transgenic).

**For this final step, it's up to you to choose which cell types to compare.** You'll also decide which pre-computed feature to compare between these cell types.

- If you'd like to compare cells from different **species**, the column name is `species`.
- If you'd like to compare **spiny vs. aspiny cells**, the column name is `dendrite_type`.
- If you'd like to compare two **transgenic lines** (mouse cells only), the column name is `transgenic_line`. What if we want to know whether different genetically-identified cells have different intrinsic physiology?
- If you'd like to compare two **disease states** (human cells only; samples taken from people either with epilepsy and brain tumors), the column name is `disease_state`.


>**Task**: Assign `column_name` below to the name of your column to see the unique values in that column. Make sure your column is a **string**, in other words, it should be in single  quotes.

In [ ]:
# Define your column name below
column_name = ...

print(full_dataframe[column_name].unique())

> Using the possible values in your column, create two separate dataframes by **subsetting** the dataframe below. Assign `celltype_1` and `celltype_2` to the names of your cell types, for example 'spiny' and 'aspiny'. Make sure your cell type names are in quotes (they should be strings) and *exactly* match what is found in the dataframe.

In [ ]:
# Define your cell type variables below
celltype_1 = ...
celltype_2 = ...

celltype_1_df = full_dataframe[full_dataframe[column_name] == celltype_1]
celltype_2_df = full_dataframe[full_dataframe[column_name] == celltype_2]

print("Type 1 # Cells: %d" % len(celltype_1_df))
print("Type 2 # Cells: %d" % len(celltype_2_df))

Let's start by plotting a distribution of the recorded resting membrane potential (`vrest`) for one cell type versuss the other cell type.

>**Task**: Run the cell below to plot a histogram to compare one pre-computedd feature of your choice between your two cell types.

- Note that the distribution is normalized by the total count (`density=True`), since there may be very different numbers of cells for your two cell types. You can set `density` to false to plot the raw numbers of cells.
- You can also specify the number of bins with `bins= < #bins > `.
- Look through the [`plt.hist()` documentation](https://matplotlib.org/3.3.1/api/_as_gen/matplotlib.pyplot.hist.html) for more information.

In [ ]:
plt.figure()

# Change your feature below.
feature = 'vrest'

# Plot the histogram, with density = True
plt.hist([celltype_1_df[feature],celltype_2_df[feature]],density = True)

# Change the x label below:
plt.xlabel('< feature name >')
plt.ylabel('Normalized Number of Cells')
plt.legend(['Cell Type 1','Cell Type 2'])
plt.show()

> <b>Task</b>: Choose a different feature to compare between your cell types, and rerun the plot above. Use the documentation below to get the exact name of the feature (in parentheses), and change the x label axis so that we know what you're plotting.  Right click to save your image when you're done.

Here are a few additional pre-computed features you might consider comparing (you can find a complete glossary [here](https://docs.google.com/document/d/1YGLwkMTebwrXd_1E817LFbztMjSTCWh83Mlp3_3ZMEo/edit?usp=sharing)):

- <b>Tau (<code>tau</code>)</b>: time constant of the membrane in milliseconds
- <b>Adapation ratio (<code>adaptation</code>)</b>: The rate at which firing speeds up or slows down during a stimulus<br>
- <b>Average ISI (<code>avg_isi</code>)</b>: The mean value of all interspike intervals in a sweep<br>
- **Slope of f/I curve** (<code>f_i_curve_slope</code>)</b>: slope of the curve between firing rate (f) and current injected<br>
- **Input Resistance** (<code>input_resistance_mohm</code>)</b>: The input resistance of the cell, in megaohms.<br>
- **Voltage of after-hyperpolarization** (<code>trough_v_short_square</code>)</b>: minimum value of the membrane potential during the after-hyperpolarization

><b>Task</b>: It's more common to plot summary statistics like a mean or median, so let's compare our two cell types with a boxplot. To do so, we can use `plt.boxplot()` ([Documentation here](https://matplotlib.org/3.3.2/api/_as_gen/matplotlib.pyplot.boxplot.html)). The code below is already set up for you -- just run it and edit your labels as necessary. Right click to save the plot when you're done.

In [ ]:
# Boxplot creation lines below
plt.boxplot([celltype_1_df[feature],celltype_2_df[feature]])
plt.ylabel('< your label here > ') # y-axis label
plt.xticks([1, 2], ['Cell Type 1','Cell Type 2'])

# Plot title -- be sure to update!
plt.title('< your title here > ')

plt.show()

That's it for today -- great work!

In [ ]:
from IPython.display import HTML
print('Nice work!')
HTML('<img src="https://media.giphy.com/media/xUOwGhOrYP0jP6iAy4/giphy.gif">')

## Extra excercies:

1: Some plotting
- select 5 cells in your favourite layer, genotype, etc.
- go through each cell in a loop, and find the first sweep when it fires an action potential
- in the same loop, make a plot of the firing pattern of every cell, and compare their firing patterns visually

2: Correlations
- go through all of the cellular feature pairs (nested loops)
- calculate the Pearson's correlation (scipy.stats.pearsonr), and continue only if significant
- plot them against each other
- do a linear fit (np.polyfit())
- plot the linear fit as a line, and add the Pearson's R as a legend (plt.plot(,label = 'r = ...'))

3: Your own question :)


<hr>
<a id="technical"></a>

## Technical notes & credits

This notebook demonstrates most of the features of the AllenSDK that help manipulate data in the Cell Types Database.  The main entry point will be through the `CellTypesCache` class. `CellTypesCache` is responsible for downloading Cell Types Database data to a standard directory structure on your hard drive.  If you use this class, you will not have to keep track of where your data lives, other than a root directory.

Much more information can be found in the <a href="http://help.brain-map.org/download/attachments/8323525/CellTypes_Ephys_Overview.pdf">Allen Brain Atlas whitepaper</a> as well as in their <a href="http://alleninstitute.github.io/AllenSDK/cell_types.html">GitHub documentation</a>.

This notebook was largely inspired by <a href='https://alleninstitute.github.io/AllenSDK/_static/examples/nb/cell_types.html'>this</a> notebook.